<font size=5  color=#003366> [LINMA2472] - Algorithms in Data Science <br>
Project 1: Networks </font> <br><br>
<font size=3  color=#003366>  Students: Iacopo Canetta, Giovanni Faldani
<br><br>

- <font size=4  color=#003366>Task 1: building the co-occurence network of characters </font> <br><br>
In this project we'll build a graph network representing the bonds between characters present in the classical novel "The Betrothed" by Alessandro Manzoni

In [11]:
# basic imports
import numpy as np
import pandas as pd # useful dataframe support
import scipy as sp
from matplotlib import pyplot as plt # plot operations
import networkx as nx # useful for network operations
import nltk # useful for sentence parsing
import re # regular expressions for text parsing and finding proper names
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Giovanni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
# text file parsing
txt_filename = "./data/pg35155.txt"

text = []
with open(txt_filename, "r") as input_f:
    text = input_f.read()

# split text by paragraph
paragraphs = text.split("\n\n")#\n is the name for skipping to the next line
print("The text is composed of {} paragraphs!\n".format(len(paragraphs)))

# extract all individual sentences from a text
def sentences_in(text):
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

    sentences = sent_detector.tokenize(text.strip())
    return sentences

# extract all the dialogue within quotations in a text
def dialogue_in(text):
    return re.findall('"([^"]*)"', text)

# extract all words beginning with a capital letter in a text
def proper_names_in(text):
    return re.findall("(?<!^)(?<!\. )[A-Z][a-z]+", text)

# test with a random paragraph
n = 149

print(paragraphs[n])
dialogues = dialogue_in(paragraphs[n])
sentences = sentences_in(paragraphs[n])
names = proper_names_in(paragraphs[n])
print("\nParagraph {} is composed of {} sentences, {} dialogues and {} proper names".format(
    n, len(sentences), len(dialogues), len(names)))
print(sentences[0])
print(dialogues[0])
print(names[0])

The text is composed of 2696 paragraphs!

"Oh! for Heaven's sake--how should I understand all this Latin?"

Paragraph 149 is composed of 2 sentences, 1 dialogues and 3 proper names
"Oh!
Oh! for Heaven's sake--how should I understand all this Latin?
Oh


- As we can see finding the proper names of characters is no easy task, so we decided to rely on an external list of characters to look for matches and build our graph.

In [31]:
# let's list all the proper names in the book. This might take a while...
allNames = set(proper_names_in(text))
print(allNames)
# need to make a decision here

{'Suffer', 'Dublin', 'Lent', 'Frenchmen', 'Hurried', 'Approach', 'Fuentes', 'Proofreading', 'Unknown', 'Special', 'Corsia', 'Bravo', 'Avola', 'Anselmo', 'Beso', 'Tadino', 'Quick', 'Dress', 'Where', 'Immediately', 'Salio', 'Lend', 'Saints', 'Reflecting', 'College', 'Capitano', 'Mixture', 'Books', 'San', 'Excepting', 'Lucy', 'Her', 'Pierpaolo', 'Heaps', 'Pollute', 'Trilingue', 'Unfeeling', 'Marco', 'Pretty', 'Since', 'Freas', 'Web', 'Adam', 'Gera', 'Jervase', 'Transcriber', 'Carlo', 'Except', 'Bione', 'Even', 'Pshaw', 'What', 'Could', 'Melan', 'Umph', 'Pavia', 'Rascals', 'Galignani', 'Satan', 'Renzo', 'Gregory', 'Sections', 'Milanese', 'Wearied', 'April', 'Aim', 'Spanish', 'Hush', 'Usted', 'Dinner', 'Rivola', 'Pentecost', 'Virgin', 'Cassano', 'Gonzalo', 'Note', 'Cupid', 'Heliogabalus', 'Paris', 'Bartolo', 'Chamber', 'Resegone', 'Kite', 'Schenchio', 'Alcabizio', 'Bergamascans', 'Atqui', 'European', 'Gently', 'Arragon', 'Section', 'Don', 'Printed', 'Sicily', 'Aroused', 'Know', 'Saturn', 'S